<a href="https://colab.research.google.com/github/UdaraChamidu/Generative-AI/blob/main/Chatbot_Langchain_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install packeges
!pip install langchain -qu
!pip install langchain-openai -qU


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.8 MB/s eta 0:00:00


In [2]:
# import necaessary libraries
import os
from google.colab import userdata # this need only we have hide our secreat keys.



# Initialize LLM

In [3]:
from langchain_openai import ChatOpenAI

# set api key as an environment variable
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# if u havent hide the api key, u can directly type api key after '='.

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0
    )




# Initialize Prompt Template

In [4]:
# to send messages
from langchain_core.prompts import ChatPromptTemplate

# create  a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an intelligent chatbot. Answer the following questions."), # a system message
        ("user","{question}")  # this is an  user message. user message is actually the question.
    ]
)

# user and system letters consider. all should lowercase.



# initialize output parser

In [5]:
from langchain_core.output_parsers import StrOutputParser

# string output parser(not essential to here)
parser = StrOutputParser()

# Make the chain

In [6]:
 chain = prompt | llm | parser

# Test using basic questions

In [7]:
question = "My name is udara !" # asking questions
response = chain.invoke({"question":question})
print(response)


Hello Udara! How can I assist you today?


# Use prompt template

In [8]:
# make some changes of the prompt
# we can give messages as a list
# system, Ai, user messages...
# we store the "question" in the place holder.

from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = "You are an intelligent chatbot. Answer the following questions."), # a system message
        MessagesPlaceholder(variable_name="question")  # can store some messages (here only question)
    ]
)

In [9]:
# again chain
chain = prompt | llm | parser

In [10]:
question = "My name is Udara .. . "
response = chain.invoke({"question": [HumanMessage(content=question)]})
print(response)

Hello Udara! How can I assist you today?


In [11]:
# above i told my name but when i asked my name again, it can not give my name as output.
# because there is not a memory
question = "who am I"
response = chain.invoke({"question": [HumanMessage(content=question)]})
print(response)

I'm sorry, but I don't have access to personal information about users. How can I assist you today?


In [12]:
# with history

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = "You are an intelligent chatbot. Answer the following questions."), # a system message
        HumanMessage(content="My name is Udara ... "),       # human message
        AIMessage(content="Nice to meet you, udara ! How can I assist you today ? "),
        MessagesPlaceholder(variable_name="question")  # can store some messages (here only question
    ]
)

In [13]:
chain = prompt | llm | parser

In [14]:
question = "who am I"
response = chain.invoke({"question": [HumanMessage(content=question)]})
print(response)

# after this code the model can memorize my name.

You are Udara, a user interacting with me right now.


In [15]:
# we can not give the codes mannually every time like above code.
#

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = "You are an intelligent chatbot. Answer the following questions."), # a system message
        MessagesPlaceholder(variable_name="history"),  # can store some messages (here only history)
        MessagesPlaceholder(variable_name="question")  # can store some messages (here only question)
    ]
)

In [16]:
# create the history (variable)
# if we dont need, we can use an empty history
history = [
    HumanMessage(content="My name is Udara ... "),       # human message
    AIMessage(content="Nice to meet you, udara ! How can I assist you today ? "),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4")
]


In [17]:
# now history has added to the prompt
chain = prompt | llm | parser

In [18]:
question = "who am I"
response = chain.invoke({"history": history, "question": [HumanMessage(content=question)]})
print(response)

# after this code the model can memorize my name. (like previous)

You are Udara.


In [19]:
history

# new things are not addded to the history.
# who am i, had not added to the history.

[HumanMessage(content='My name is Udara ... ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, udara ! How can I assist you today ? ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={})]

In [20]:
# extend, add the latest message and response to the history

history.extend([HumanMessage(content=question), AIMessage(content=response)])

In [21]:
history

[HumanMessage(content='My name is Udara ... ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, udara ! How can I assist you today ? ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='who am I', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are Udara.', additional_kwargs={}, response_metadata={})]

In [22]:
question = "what is my last question ?"
response = chain.invoke({"history": history,
                         "question": [HumanMessage(content=question)]}
)
history.extend([HumanMessage(content=question), AIMessage(content=response)])
# extend the human and AI messeages at once when they comes ...
print(response)

Your last question was "who am I".


In [23]:
# display the last 4 interactions... (last 4 messages)
history[-4:]

[HumanMessage(content='who am I', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are Udara.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is my last question ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your last question was "who am I".', additional_kwargs={}, response_metadata={})]